In [52]:
#This will be the script that handles the main game of RPS
import random
import numpy as np
import pandas as pd


In [76]:

# ask for player name and how many rounds they want to play
player_id = 'Ethan Fahimi' #input("What is your name?")
rounds_tot = 20 #int(input("How many rounds would you like to play?"))



In [88]:
# Create game logic
outcomes = {'0':{'0':0,'1':-1,'2':1,'-1':-2},
            '1':{'0':1,'1':0,'2':-1,'-1':-2},
            '2':{'0':-1,'1':1,'2':0,'-1':-2},
            '-1':{'0':-2,'1':-2,'2':-2,'-1':-2}}

what_beats_this = {'0':1,
                   '1':2,
                   '2':0}

what_loses_this = {'0':2,
                   '1':0,
                   '2':1}

outcome_text = {'0': '\nYou Tied. :| \n',
                '1': '\nYou Won! :)))\n',
                '-1': '\nYou Lose... :(\n'}

scores = [[]]
columns = ['user','round_num','player_choice','cpu_choice','outcome','model_used','model_choices','model_outcomes','model_scores']

In [11]:
# model 0 - simple model based on rough probability of human choice
def model_0():
    model_0_prob = random.random()
    model_0_pred = 1
    if model_0_prob < 0.5: # 50% chance to throw paper
        model_0_pred = 1
    elif model_0_prob > .8: # 20% chance to throw scissors
        model_0_pred = 2
    else:
        model_0_pred = 0 # 30% chance to throw rock
    return model_0_pred

In [37]:
# model 1 - simple model that plays what would beat the opponent
# based on the round robin pattern (0->1->2->0 or 2->1->0->2)

def model_1():
    model_1_pred = -1 # assigning this a value for rounds that it will be unused
    if Round == 1: # guess what would beat the previous round
        model_1_pred = what_beats_this[str(history['player_choice'][Round - 1])]
    if Round > 1: # we want two rounds to have been played before we can see the pattern
        previous_choice = history['player_choice'][Round - 1]
        previous_previous_choice = history['player_choice'][Round - 2]
        change_in_choice = previous_choice - previous_previous_choice
        
        model_1_player_pred = previous_choice + change_in_choice # predict what they will play
        
        # make sure prediction is within bounds [0,2]
        if model_1_player_pred > 2:
            model_1_player_pred -= 3
        elif model_1_player_pred < 0:
            model_1_player_pred += 3
        
        # assign prediction of what will beat them
        model_1_pred = what_beats_this[str(model_1_player_pred)]
        
    return model_1_pred

In [82]:
# model 2 - simple model that tries to guess if the player is alternating between choices, i.e. 0->1->0->1->0

def model_2():
    model_2_pred = -1 # assigning this a value for rounds that it will be unused
    if Round > 1: # two rounds must have been played before we can pick up this pattern - similar to model_1
        previous_choice = history['player_choice'][Round - 1]
        previous_previous_choice = history['player_choice'][Round - 2]
        change_in_choice = previous_choice - previous_previous_choice
        # positive change in choice means next alternate is a negative change in choice
        if change_in_choice > 0:
            sign = -1
        elif change_in_choice < 0:
            sign = 1
        else:
            sign = 0
            
        model_2_player_pred = previous_choice + sign
        
        # make sure prediction is within bounds [0,2]
        if model_2_player_pred > 2:
            model_2_player_pred -= 3
        elif model_2_player_pred < 0:
            model_2_player_pred += 3
        
        # assign prediction of what will beat them
        model_2_pred = what_beats_this[str(model_2_player_pred)]
        
    return model_2_pred

In [80]:
# model 3 - simple model that plays what would beat the opponents most played move in past 7 moves

def model_3():
    model_3_pred = -1 # assigning this a value for rounds that it will be unused
    if Round > 7:
        # tally up most used move in past 7 rounds
        choice_tally = [0,0,0]
        for i in range(Round-8,Round-1):
            choice_tally[history['player_choice'][i]] += 1
        model_3_pred = what_beats_this[str(np.argmax(choice_tally))] # choose what would beat the most common move
    return model_3_pred

In [70]:
def ensembler():
    round_score = []
    for i in range(len(history['model_outcomes'][0])):
        model_sqr_sum = 1
        model_outcome_sum = 0
        for j in range(len(history['model_outcomes'])):
            if history['model_outcomes'][j][i] == -2:
                pass
            else:
                model_sqr_sum += (j+1)**2
                model_outcome_sum += history['model_outcomes'][j][i] *((j+1)**2)
        round_score.append(model_outcome_sum / model_sqr_sum)
    return round_score

In [89]:
history = pd.DataFrame(columns=columns) # create history df
Round = 0
while Round < rounds_tot:
    # call models and create choice list
    choice_list = [model_0(),model_1(),model_2(),model_3()]
    if Round == 0:
        selected_model = 0
        round_score = [0.0,0.0,0.0,0.0]
    else:
        round_score = ensembler()
        selected_model = np.argmax(round_score)
        while choice_list[selected_model] == -1:
            selected_model -= 1
        scores.append(round_score)
    selected_choice = choice_list[selected_model]
    print(round_score)
    #have player select choice
    player_choice = int(input("Please select Rock (0), Paper (1), or Scissors (2)"))

    # determine game outcome
    outcome = outcomes[str(player_choice)][str(selected_choice)]
    
    # determine model outcomes
    model_outcomes = [outcomes[str(x)][str(player_choice)] for x in choice_list]
    
    # make important printouts for player
    print(selected_choice)
    print(outcome_text[str(outcome)])
    
    # save important data from the round
    history = history.append({'user':player_id,
                              'round_num':Round,
                              'player_choice':player_choice,
                              'cpu_choice':selected_choice,
                              'outcome':outcome,
                              'model_used':selected_model,
                              'model_choices':choice_list,
                              'model_outcomes':model_outcomes,
                              'model_scores':round_score}, 
                             ignore_index=True)
    print(history.tail())
    Round += 1
    

[0.0, 0.0, 0.0, 0.0]
Please select Rock (0), Paper (1), or Scissors (2)0
1

You Lose... :(

           user round_num player_choice cpu_choice outcome model_used  \
0  Ethan Fahimi         0             0          1      -1          0   

     model_choices   model_outcomes          model_scores  
0  [1, -1, -1, -1]  [1, -2, -2, -2]  [0.0, 0.0, 0.0, 0.0]  
[0.5, 0.0, 0.0, 0.0]
Please select Rock (0), Paper (1), or Scissors (2)1
1

You Tied. :| 

           user round_num player_choice cpu_choice outcome model_used  \
0  Ethan Fahimi         0             0          1      -1          0   
1  Ethan Fahimi         1             1          1       0          0   

     model_choices   model_outcomes          model_scores  
0  [1, -1, -1, -1]  [1, -2, -2, -2]  [0.0, 0.0, 0.0, 0.0]  
1   [1, 1, -1, -1]   [0, 0, -2, -2]  [0.5, 0.0, 0.0, 0.0]  
[0.16666666666666666, 0.0, 0.0, 0.0]
Please select Rock (0), Paper (1), or Scissors (2)0
1

You Lose... :(

           user round_num player_choice cp

KeyError: '3'

In [ ]:
history_pred = [[1,0,0,1,2,2,0,1],[1,0,0,1,2,2,0,1],[1,0,0,1,2,2,0,1],[1,0,0,1,2,2,0,1]]
history_outcome = [[0,-1,-1,1,0,0,1,-1],[0,-1,-1,1,0,0,1,-1],[0,-1,-1,1,0,0,1,-1],[0,-1,-1,1,0,0,1,-1]]


print(len(history['model_choices'][0]))

In [33]:
print(len(history['model_choices']))
print(len(history['model_choices'][0]))

1
2


In [50]:
model_outcomes = [outcomes[str(player_choice)][str(x)] for x in choice_list]
print(model_outcomes)
print(choice_list)
print(player_choice)

[0, -1]
[0, 1]
0


In [67]:
print(history.tail(5))

           user round_num player_choice cpu_choice outcome model_used  \
0  Ethan Fahimi         0             0          1      -1          0   
1  Ethan Fahimi         1             1          1       0          1   

  model_choices model_outcomes     model_scores  
0   [1, -1, -1]   [-1, -2, -2]        [0, 0, 0]  
1    [2, 1, -1]    [-1, 0, -2]  [0.0, 1.0, 1.0]  
